Author: Bram van Duinen

Date last edited: 31 January 2024

Content: Script that uses as input running mean calculated per run (calculated using cdo in running_mean.sh shell script). In this script, those are averaged over all runs and then saved to one netcdf. Now also updated to do the same for zg500 climatology.

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
# base_zg500 = '/net/pc230050/nobackup/users/duinen/LENTIS/present/climatology/zg500_d/'
# zg500 = xr.open_mfdataset(base_zg500+'zg500_d_ECEarth3_h*.nc', concat_dim='run', combine="nested")
# clim  = zg500.zg500.mean(dim='run').compute()
# clim['time'] = np.arange(1, 367)
# clim = clim.rename({'time': 'dayofyear'})
# clim.to_netcdf(f'../Data/clim_zg500_v1.nc')

In [34]:
base_precip = '/net/pc230050/nobackup/users/duinen/LENTIS/present/pr_d_ydaymean/'
precip = xr.open_mfdataset(base_precip+'pr_d_ECEarth3_h*.nc', concat_dim='run', combine="nested")
clim_precip = precip.isel(run = 0)
clim_precip['pr']  = precip.pr.mean(dim='run').compute()
clim_precip_mm = clim_precip * 86400 # convert to mm/day
clim_precip_mm.to_netcdf(f'../Data/clim_precip_mm_v1.nc')

In [54]:
precip_sum = xr.open_dataset(base_precip + 'pr_d_ECEarth3_h010.nc')
for run in range(11, 170):
    file_i = xr.open_dataset(base_precip + f'pr_d_ECEarth3_h{run:03d}.nc')
    if len(file_i.lat) != 68:
        print(f'Run {run} has {len(file_i.lat)} latitudes')
        continue 
    precip_sum['pr'] += file_i.pr

MergeError: Automatic alignment is not supported for in-place operations.
Consider aligning the indices manually or using a not-in-place operation.
See https://github.com/pydata/xarray/issues/3910 for more explanations.

In [53]:
len(xr.open_dataset(base_precip + 'pr_d_ECEarth3_h010.nc').lat)

68

In [50]:
precip_files.isel(run = 0)

<xarray.Dataset>
Dimensions:   (time: 366, lon: 155, lat: 70, bnds: 2)
Coordinates:
  * time      (time) datetime64[ns] 2009-01-01T12:00:00 ... 2009-12-31T12:00:00
  * lon       (lon) float64 -32.34 -31.64 -30.94 -30.23 ... 74.53 75.23 75.94
  * lat       (lat) float64 33.33 34.04 34.74 35.44 ... 78.25 78.95 79.65 80.35
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float64 -32.7 -31.99 -31.99 ... 75.59 75.59 76.29
    lat_bnds  (lat, bnds) float64 32.98 33.68 33.68 34.39 ... 80.0 80.0 80.7
    pr        (time, lat, lon) float32 5.153e-05 5.8e-05 ... 7.572e-06 7.219e-06
Attributes: (12/52)
    CDI:                                Climate Data Interface version 2.4.0 ...
    Conventions:                        CF-1.7 CMIP-6.2
    source:                             EC-Earth3 (2019): \naerosol: none\nat...
    institution:                        The Royal Netherlands Meteorological ...
    activity_id:                        CMIP
    branch_method:                      standard
    ...                                 ...
    variable_id:                        pr
    variant_info:                       The p5 label refers to the fact that ...
    variant_label:                      r1i1p5f1
    license:                            The VAREX model data produced by KNMI...
    cmor_version:                       3.5.0
    CDO:                                Climate Data Operators version 2.4.0 ...

In [46]:
precip_v2 = xr.open_mfdataset(base_precip+'pr_d_ECEarth3_h*.nc', combine="by_coords")

In [47]:
precip_v2

<xarray.Dataset>
Dimensions:   (time: 366, lon: 155, lat: 68, bnds: 2)
Coordinates:
  * time      (time) datetime64[ns] 2009-01-01T12:00:00 ... 2009-12-31T12:00:00
  * lon       (lon) float64 -32.34 -31.64 -30.94 -30.23 ... 74.53 75.23 75.94
  * lat       (lat) float64 33.33 34.04 34.74 35.44 ... 78.25 78.95 79.65 80.35
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lat, lon, bnds) float64 dask.array<chunksize=(68, 155, 2), meta=np.ndarray>
    lat_bnds  (lat, bnds) float64 dask.array<chunksize=(68, 2), meta=np.ndarray>
    pr        (time, lat, lon) float32 dask.array<chunksize=(1, 68, 155), meta=np.ndarray>
Attributes: (12/52)
    CDI:                                Climate Data Interface version 2.4.0 ...
    Conventions:                        CF-1.7 CMIP-6.2
    source:                             EC-Earth3 (2019): \naerosol: none\nat...
    institution:                        The Royal Netherlands Meteorological ...
    activity_id:                        CMIP
    branch_method:                      standard
    ...                                 ...
    variable_id:                        pr
    variant_info:                       The p5 label refers to the fact that ...
    variant_label:                      r1169i9p5f1
    license:                            The VAREX model data produced by KNMI...
    cmor_version:                       3.5.0
    CDO:                                Climate Data Operators version 2.4.0 ...

In [24]:
# base_precip = '/net/pc230050/nobackup/users/duinen/LENTIS/present/pr_d_ydaymean/'
# precip = xr.open_mfdataset(base_precip+'pr_d_ECEarth3_h*.nc', concat_dim='run', combine="nested")
# clim_precip = precip.isel(run = 0)
# clim_precip['pr']  = precip.pr.mean(dim='run').compute()
# clim_precip['time'] = pd.date_range(start='2004-01-01', end='2004-12-31')
# # clim_precip = clim_precip.rename({'time': 'dayofyear'})
# clim_precip_mm = clim_precip * 86400 # convert to mm/day
# clim_precip_mm.to_netcdf(f'../Data/clim_precip_mm_v1.nc')

In [43]:
xr.open_dataset(base_precip + 'pr_d_ECEarth3_h010.nc')

<xarray.Dataset>
Dimensions:   (time: 366, lon: 155, bnds: 2, lat: 68)
Coordinates:
  * time      (time) datetime64[ns] 2009-01-01T12:00:00 ... 2009-12-31T12:00:00
  * lon       (lon) float64 -32.34 -31.64 -30.94 -30.23 ... 74.53 75.23 75.94
  * lat       (lat) float64 33.33 34.04 34.74 35.44 ... 78.25 78.95 79.65 80.35
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float64 ...
    lat_bnds  (lat, bnds) float64 ...
    pr        (time, lat, lon) float32 ...
Attributes: (12/52)
    CDI:                                Climate Data Interface version 2.4.0 ...
    Conventions:                        CF-1.7 CMIP-6.2
    source:                             EC-Earth3 (2019): \naerosol: none\nat...
    institution:                        The Royal Netherlands Meteorological ...
    activity_id:                        CMIP
    branch_method:                      standard
    ...                                 ...
    variable_id:                        pr
    variant_info:                       The p5 label refers to the fact that ...
    variant_label:                      r1i1p5f1
    license:                            The VAREX model data produced by KNMI...
    cmor_version:                       3.5.0
    CDO:                                Climate Data Operators version 2.4.0 ...

In [41]:
clim_precip_mm.lat

<xarray.DataArray 'lat' (lat: 70)>
array([33.333262, 34.035014, 34.736767, 35.438519, 36.140271, 36.140271,
       36.842024, 37.543776, 38.245528, 38.94728 , 39.649033, 40.350785,
       41.052537, 41.754289, 42.456041, 43.157793, 43.859545, 44.561296,
       45.263048, 45.263048, 45.9648  , 46.666551, 47.368303, 48.070054,
       48.771806, 49.473557, 50.175308, 50.877059, 51.57881 , 52.280561,
       52.982312, 53.684062, 54.385813, 55.087563, 55.789314, 56.491064,
       57.192814, 57.894563, 58.596313, 59.298062, 59.999811, 60.70156 ,
       61.403309, 62.105058, 62.806806, 63.508554, 64.210301, 64.912048,
       65.613795, 66.315541, 67.017287, 67.719033, 68.420777, 69.122522,
       69.824265, 70.526008, 71.22775 , 71.929491, 72.631231, 73.33297 ,
       74.034707, 74.736443, 75.438177, 76.13991 , 76.84164 , 77.543367,
       78.245091, 78.946811, 79.648527, 80.350237])
Coordinates:
  * lat      (lat) float64 33.33 34.04 34.74 35.44 ... 78.25 78.95 79.65 80.35
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north
    axis:           Y
    bounds:         lat_bnds

In [40]:
xr.open_dataset('/net/pc230050/nobackup/users/duinen/LENTIS/present/pr_d/pr_d_ECEarth3_h010.nc').lat

<xarray.DataArray 'lat' (lat: 68)>
array([33.333262, 34.035014, 34.736767, 35.438519, 36.140271, 36.842024,
       37.543776, 38.245528, 38.94728 , 39.649033, 40.350785, 41.052537,
       41.754289, 42.456041, 43.157793, 43.859545, 44.561296, 45.263048,
       45.9648  , 46.666551, 47.368303, 48.070054, 48.771806, 49.473557,
       50.175308, 50.877059, 51.57881 , 52.280561, 52.982312, 53.684062,
       54.385813, 55.087563, 55.789314, 56.491064, 57.192814, 57.894563,
       58.596313, 59.298062, 59.999811, 60.70156 , 61.403309, 62.105058,
       62.806806, 63.508554, 64.210301, 64.912048, 65.613795, 66.315541,
       67.017287, 67.719033, 68.420777, 69.122522, 69.824265, 70.526008,
       71.22775 , 71.929491, 72.631231, 73.33297 , 74.034707, 74.736443,
       75.438177, 76.13991 , 76.84164 , 77.543367, 78.245091, 78.946811,
       79.648527, 80.350237])
Coordinates:
  * lat      (lat) float64 33.33 34.04 34.74 35.44 ... 78.25 78.95 79.65 80.35
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north
    axis:           Y
    bounds:         lat_bnds